In [21]:
import os
import pandas as pd
import numpy as np
from scripts.sample_db import SampleDB
from skimage import measure
import tifffile
from datetime import datetime
import csv

def create_bigwarp_csv(filename, landmarks, moving="EM", fixed="LM"):
    columns = ['Point_id', 'Active', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z']
    df = pd.DataFrame(columns=columns)
    for i, landmark in enumerate(landmarks):
        point_id = f"Pt-{i}"
        if moving == "EM":
            row = [point_id, "false"] + list(landmark) + ["Infinity", "Infinity", "Infinity"]
        else:
            row = [point_id, "false", "Infinity", "Infinity", "Infinity"] + list(landmark)
        df.loc[len(df)] = row
    df.to_csv(filename, index=False, header=None, quoting=csv.QUOTE_NONNUMERIC)

def split_unmatched_landmarks(unmatched_landmarks, base_filename, max_landmarks=3000):
    num_files = (len(unmatched_landmarks) + max_landmarks - 1) // max_landmarks
    
    for i in range(num_files):
        start_index = i * max_landmarks
        end_index = min(start_index + max_landmarks, len(unmatched_landmarks))
        landmarks_chunk = unmatched_landmarks[start_index:end_index]

        df = pd.DataFrame(landmarks_chunk, columns=['moving_x', 'moving_y', 'moving_z'])
        filename = f'{base_filename}_part_{i + 1}.csv'
        df.to_csv(filename, index=False)
        print(f'Saved {filename} with {len(landmarks_chunk)} landmarks.')

def create_fine_landmarks(rough_landmarks_file, unmatched_landmarks, chunk_size=3000):
    now = datetime.now()
    formatted_datetime = now.strftime("%Y%m%d_%H%M%S")
    rough_df = pd.read_csv(rough_landmarks_file, header=None)
    existing_landmarks = rough_df[rough_df.iloc[:, 1] == 'true'].iloc[:, 2:8].values.tolist()
    
    unmatched_landmarks_formatted = [[landmark[0], landmark[1], landmark[2], 'Infinity', 'Infinity', 'Infinity'] for landmark in unmatched_landmarks]
    
    columns = ['Point_id', 'matched', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z']
    fine_df = pd.DataFrame(columns=columns)
    
    # Add existing landmarks to fine_df
    for i, landmark in enumerate(existing_landmarks):
        point_id = f'Pt-{i}'
        row = [point_id, 'true'] + landmark
        fine_df.loc[len(fine_df)] = row
    
    # Process unmatched landmarks in chunks
    num_chunks = (len(unmatched_landmarks_formatted) + chunk_size - 1) // chunk_size
    for i in range(num_chunks):
        batch_landmarks = unmatched_landmarks_formatted[i * chunk_size:(i + 1) * chunk_size]
        for j, landmark in enumerate(batch_landmarks):
            point_id = f'Pt-{len(existing_landmarks) + i * chunk_size + j}'
            row = [point_id, 'false'] + landmark
            fine_df.loc[len(fine_df)] = row
        
        filename = f'{formatted_datetime}_fine_landmarks_until_part_{i + 1}.csv'
        fine_df.to_csv(filename, index=False, header=None, quoting=csv.QUOTE_NONNUMERIC)
        print(f'Saved fine landmarks with name {filename}.')
    
    return filename

def update_fine_landmarks(fine_landmarks_file, unmatched_landmarks, chunk_size=3000):
    fine_df = pd.read_csv(fine_landmarks_file, header=None)
    active_landmarks = fine_df[fine_df.iloc[:, 1] == 'true']
    existing_landmarks_count = len(active_landmarks)
    
    unmatched_landmarks_formatted = [[landmark[0], landmark[1], landmark[2], 'Infinity', 'Infinity', 'Infinity'] for landmark in unmatched_landmarks]
    
    new_fine_df = pd.DataFrame(columns=fine_df.columns)
    new_fine_df = pd.concat([new_fine_df, active_landmarks])
    
    # Process unmatched landmarks in chunks
    num_chunks = (len(unmatched_landmarks_formatted) + chunk_size - 1) // chunk_size
    for i in range(num_chunks):
        batch_landmarks = unmatched_landmarks_formatted[i * chunk_size:(i + 1) * chunk_size]
        for j, landmark in enumerate(batch_landmarks):
            point_id = f'Pt-{existing_landmarks_count + i * chunk_size + j}'
            row = [point_id, 'false'] + landmark
            new_fine_df.loc[len(new_fine_df)] = row
        
        part_number = int(fine_landmarks_file.split('_part_')[-1].split('.')[0]) + i + 1
        new_filename = fine_landmarks_file.replace(f'until_part_{part_number - i - 1}', f'until_part_{part_number}')
        new_fine_df.to_csv(new_filename, index=False, header=None, quoting=csv.QUOTE_NONNUMERIC)
        print(f'Created new file {new_filename} with active landmarks and next batch of unmatched landmarks.')
    
    return new_filename

def read_active_landmarks(csv_file):
    """
    Reads a CSV file and returns only the active landmarks.
    
    Args:
    csv_file (str): Path to the CSV file.
    
    Returns:
    pandas.DataFrame: DataFrame containing only the active landmarks.
    """
    columns = ['Point_id', 'matched', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z']
    df = pd.read_csv(csv_file, header=None, names=columns)
    active_landmarks = df[df['matched'] == 'true']
    return active_landmarks

def merge_landmarks_to_csv(active_landmarks, unmatched_landmarks_file, output_file):
    """
    Merges active landmarks with unmatched landmarks into a new CSV file,
    keeping previously active landmarks as active.
    
    Args:
    active_landmarks (pandas.DataFrame): DataFrame containing active landmarks.
    unmatched_landmarks_file (str): Path to the CSV file with unmatched landmarks.
    output_file (str): Path where the merged CSV file will be saved.
    
    Returns:
    str: Path to the output file.
    """
    columns = ['Point_id', 'matched', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z']
    unmatched_df = pd.read_csv(unmatched_landmarks_file, header=0)
    
    # Prepare unmatched landmarks
    unmatched_landmarks = pd.DataFrame(columns=columns)
    unmatched_landmarks['Point_id'] = [f'Pt-{i+len(active_landmarks)}' for i in range(len(unmatched_df))]
    unmatched_landmarks['matched'] = 'false'
    unmatched_landmarks['moving_x'] = unmatched_df['moving_x']
    unmatched_landmarks['moving_y'] = unmatched_df['moving_y']
    unmatched_landmarks['moving_z'] = unmatched_df['moving_z']
    unmatched_landmarks['fixed_x'] = 'Infinity'
    unmatched_landmarks['fixed_y'] = 'Infinity'
    unmatched_landmarks['fixed_z'] = 'Infinity'
    
    # Ensure active landmarks remain active
    active_landmarks['matched'] = 'true'
    
    # Merge active and unmatched landmarks
    merged_df = pd.concat([active_landmarks, unmatched_landmarks], ignore_index=True)
    
    # Save to CSV
    merged_df.to_csv(output_file, index=False, header=False, quoting=csv.QUOTE_NONNUMERIC)
    return output_file

def create_fine_landmarks_file(rough_landmarks_file, unmatched_landmarks_file, output_path):
    # Read the rough landmarks file and extract only active landmarks
    rough_df = pd.read_csv(rough_landmarks_file, header=None, 
                           names=['Point_id', 'Active', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z'])
    active_landmarks = rough_df[rough_df['Active'] == 'true'].copy()

    # Read the unmatched landmarks file
    unmatched_df = pd.read_csv(unmatched_landmarks_file)

    # Create the fine landmarks DataFrame, starting with active landmarks
    fine_df = active_landmarks

    # Add unmatched landmarks to the file
    start_index = len(fine_df)
    for i, landmark in unmatched_df.iterrows():
        new_point_id = f'Pt-{start_index + i}'
        new_row = pd.DataFrame({
            'Point_id': [new_point_id],
            'Active': ['false'],
            'moving_x': [landmark['moving_x']],
            'moving_y': [landmark['moving_y']],
            'moving_z': [landmark['moving_z']],
            'fixed_x': ['Infinity'],
            'fixed_y': ['Infinity'],
            'fixed_z': ['Infinity']
        })
        fine_df = pd.concat([fine_df, new_row], ignore_index=True)

    # Create output filename with timestamp
    now = datetime.now()
    formatted_datetime = now.strftime('%Y%m%d_%H%M%S')
    output_file = os.path.join(output_path, f'{formatted_datetime}_fine_landmarks.csv')

    # Save the data to CSV
    fine_df.to_csv(output_file, index=False, header=False, quoting=csv.QUOTE_NONNUMERIC)
    return output_file

def main():
    # Load the sample database
    db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
    sample_db = SampleDB()
    sample_db.load(db_path)

    # Get the sample
    exp = sample_db.get_sample('20220511_RM0008_126hpf_fP10_f2')

    # Update EM paths
    em_stack_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15.tif'
    em_mask_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_cp_masks_ft04_cp-3.tif'

    exp.em_stack_path = em_stack_path
    exp.em_mask_path = em_mask_path

    # Save the updated database
    sample_db.save(db_path)
    print(f"Sample database updated and saved to {db_path}")

    # Load EM mask and calculate centroids
    em_mask = tifffile.imread(em_mask_path)
    em_props = measure.regionprops(em_mask)
    em_centroids = np.array([prop.centroid for prop in em_props])

    # Create BigWarp CSV files
    bigwarp_path = os.path.join(exp.paths.root_path, "bigwarp_alignment")
    os.makedirs(bigwarp_path, exist_ok=True)

    new_landmarks = em_centroids[:, ::-1]  # Flip coordinates to match coordinate sequence of BigWarp
    selected_indices = np.random.choice(len(em_centroids), len(em_centroids), replace=False)
    selected_centroids = new_landmarks[selected_indices].astype(str)
    new_landmarks_rnd = selected_centroids.tolist()

    # Split unmatched landmarks
    split_unmatched_landmarks(new_landmarks_rnd, os.path.join(bigwarp_path, f'unmatched_landmarks_mov_EM_fix_LM_{exp.sample.id}'))

    # Create and save centroids lookup table
    centroid_list = [tuple(centroid) for centroid in em_centroids]
    centroids_lut = pd.DataFrame({
        'id': range(1, len(centroid_list) + 1),
        'em_centroids_zyx_px': centroid_list
    })
    centroids_lut.to_csv(os.path.join(bigwarp_path, 'lut_centroids.csv'), index=False)

    print("All files saved successfully.")

    # Define file paths
    rough_landmarks_file = os.path.join(bigwarp_path, f'rough_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv')
    fine_landmarks_file = os.path.join(bigwarp_path, f'fine_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv')

    # Check if rough landmarks files exist
    if os.path.exists(rough_landmarks_file):
        print("Rough landmarks files found. Creating fine landmarks files...")
        fine_landmarks_file = create_fine_landmarks(rough_landmarks_file, new_landmarks_rnd, chunk_size=3000)
        print("Initial fine landmarks files created successfully.")
        
        # Update fine landmarks with remaining unmatched landmarks
        remaining_landmarks = new_landmarks_rnd[3000:]
        while remaining_landmarks:
            fine_landmarks_file = update_fine_landmarks(fine_landmarks_file, remaining_landmarks, chunk_size=3000)
            remaining_landmarks = remaining_landmarks[3000:]
        
        print("All fine landmarks files updated successfully.")
    else:
        print("Rough landmarks files not found. Please create rough landmarks files first.")



In [ ]:
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Get the sample
exp = sample_db.get_sample('20220511_RM0008_126hpf_fP10_f2')

bigwarp_path = os.path.join(exp.paths.root_path, "bigwarp_alignment")

In [ ]:
from skimage import io
em_stack_resliced_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\Reslice of fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15.tif'
em_stack_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15.tif'
em_mask_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_cp_masks_ft04_cp-3.tif'
em_mask = tifffile.imread(em_mask_path)

resliced_mask = np.transpose(em_mask, (1, 2, 0))

rotated_mask = np.rot90(resliced_mask, k=1, axes=(1, 2))


import napari
viewer = napari.Viewer()
viewer.add_image(tifffile.imread(em_stack_resliced_path), name='em_stack resliced')
viewer.add_image(tifffile.imread(em_stack_path), name='em_stack')
viewer.add_labels(em_mask, name="em_mask")
viewer.add_labels(resliced_mask, name='resliced_mask')

viewer.add_labels(rotated_mask, name='rotated_mask')



In [119]:
io.imsave('resliced_fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_cp_masks_ft04_cp-3.tif', rotated_mask.astype(np.uint8))
em_props = measure.regionprops(rotated_mask)
em_centroids = np.array([prop.centroid for prop in em_props])




In [100]:
def read_landmarks_csv(file_path):
    df = pd.read_csv(file_path, header=None, 
                     names=['Point_id', 'Active', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z'],
                     quoting=csv.QUOTE_ALL,
                     dtype={'Point_id': str, 'Active': str, 'moving_x': str, 'moving_y': str, 'moving_z': str, 
                            'fixed_x': str, 'fixed_y': str, 'fixed_z': str})
    return df

def save_merged_landmarks(df1, df2, filename):
    last_point_id = df1['Point_id'].str.extract('(\d+)').astype(int).max()
    # Update all Point_ids by adding (last_index + 1)
    df2['Point_id'] = 'Pt-' + (df2['Point_id'].str.extract('(\d+)').astype(int) + last_point_id + 1).astype(str)
    
    merged_landmarks = pd.concat([df1, df2])
    merged_landmarks.to_csv(filename, index=False, header=False, quoting=csv.QUOTE_ALL)

def chunk_unmatched_landmarks(unmatched_landmarks_path, chunk_size=3000):
    chunks = pd.read_csv(unmatched_landmarks_path, chunksize=chunk_size, header=None, 
                     names=['Point_id', 'Active', 'moving_x', 'moving_y', 'moving_z', 'fixed_x', 'fixed_y', 'fixed_z'],
                     quoting=csv.QUOTE_ALL,
                     dtype={'Point_id': str, 'Active': str, 'moving_x': str, 'moving_y': str, 'moving_z': str, 
                            'fixed_x': str, 'fixed_y': str, 'fixed_z': str})
    for i, chunk in enumerate(chunks):
        filename = os.path.join(bigwarp_path, f'unmatched_landmarks_chunk_{i}.csv')
        chunk.to_csv(filename, index=False, header=None, quoting=csv.QUOTE_ALL)

In [101]:

unmatched_landmarks_path = os.path.join(bigwarp_path, f'unmatched_landmarks_mov_EM_fix_LM_20220511_RM0008_126hpf_fP10_f2.csv')

chunk_unmatched_landmarks(unmatched_landmarks_path)

old_landmarks_path = os.path.join(bigwarp_path, 'rough_landmarks_mov_EM_fix_LM_20220511_RM0008_126hpf_fP10_f2.csv')
n = 0
new_landmarks_path = os.path.join(bigwarp_path, f'unmatched_landmarks_chunk_{n}.csv')

old_landmarks = read_landmarks_csv(old_landmarks_path)
new_landmarks = read_landmarks_csv(new_landmarks_path)
unmatched_landmarks = read_landmarks_csv(unmatched_landmarks_path)

In [140]:

# Reslice from the top (for 2D points)
def reslice_centroids(centroids):
    return centroids[:, [2, 1, 0]]  # Swap z and x

# Flip vertically
def flip_centroids(centroids):
    centroids[:, 1] = np.max(centroids[:, 1]) - centroids[:, 1]
    return centroids

# Rotate 90 degrees clockwise
def rotate_centroids(centroids):
    centroids[:, [0, 1]] = centroids[:, [1, 0]]
    centroids[:, 0] = np.max(centroids[:, 0]) - centroids[:, 0]
    return centroids


In [143]:
lmks_np = old_landmarks[["moving_z","moving_y","moving_x"]].to_numpy().astype(float)

# Apply transformations
lmks_np_resliced = reslice_centroids(lmks_np)
#lmks_np_flipped = flip_centroids(lmks_np_resliced.copy())
lmks_np_rotated = rotate_centroids(lmks_np_resliced.copy())

viewer = napari.Viewer()
viewer.add_labels(em_mask)
viewer.add_points(lmks_np)
viewer.add_points(lmks_np_rotated, face_color="red")

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Points layer 'lmks_np_rotated' at 0x239dabb3610>

In [102]:
old_landmarks.head(50)
old_landmarks[old_landmarks["Active"]=="true"].head(50)

,Point_id,Active,moving_x,moving_y,moving_z,fixed_x,fixed_y,fixed_z
18,Pt-18,true,1085.486985499759,610.4094180559205,1016.1697286322245,388.4023435376491,289.79276164404826,171.21611879414303
19,Pt-19,true,986.9285874679795,540.2412495073365,1005.6798956586097,338.41627829192015,266.5037994272882,172.0681539971952
24,Pt-24,true,962.5529457550401,594.4068245949627,635.0137300693686,304.09514991967995,250.452888955669,30.308560812374875
25,Pt-25,true,1176.4758215894653,593.3683640326597,728.4751806766418,410.9432178764186,250.0419974384443,49.45156034059431
26,Pt-26,true,1063.2836202984342,592.3299034703566,675.513691999187,348.47819353247917,249.63104333091835,38.76675354492042
27,Pt-27,true,1179.5912032763738,605.8298907802961,831.2827763446422,442.1757300483877,245.93239377288393,100.99266730683249
28,Pt-28,true,502.7780105396195,592.9278727043096,783.7445082791912,147.5965791684228,295.8251355373686,108.76374520325992
29,Pt-29,true,505.52254983696065,579.4186124951516,705.3627733927867,142.2819860813342,286.9501281049305,84.42254894268751
30,Pt-30,true,396.3354656784549,531.7774682324816,664.4864653796593,92.32833053747837,264.3904126980279,75.29123556370305
31,Pt-31,true,872.2571670983498,641.8112485288688,768.175229156132,279.7888228472168,284.8015837804635,81.73686853710407


In [103]:
new_landmarks.head(50)

,Point_id,Active,moving_x,moving_y,moving_z,fixed_x,fixed_y,fixed_z
0,Pt-0,false,771.75,506.3863636363636,370.3863636363636,Infinity,Infinity,Infinity
1,Pt-1,false,997.8,429.1,300.75,Infinity,Infinity,Infinity
2,Pt-2,false,836.2909700598802,1337.7516167664671,459.71382035928144,Infinity,Infinity,Infinity
3,Pt-3,false,894.59375,810.125,250.375,Infinity,Infinity,Infinity
4,Pt-4,false,906.0117785630154,429.8479387514723,463.68786808009423,Infinity,Infinity,Infinity
5,Pt-5,false,688.1746031746031,1262.031746031746,316.85714285714283,Infinity,Infinity,Infinity
6,Pt-6,false,728.831998212747,986.5829636485495,51.090064787923275,Infinity,Infinity,Infinity
7,Pt-7,false,1231.8165374677003,787.8666666666667,382.0320413436693,Infinity,Infinity,Infinity
8,Pt-8,false,247.32258064516128,622.258064516129,347.06451612903226,Infinity,Infinity,Infinity
9,Pt-9,false,644.4129217142113,1312.753071946507,713.7413486170814,Infinity,Infinity,Infinity


In [104]:
unmatched_landmarks.head(50)


,Point_id,Active,moving_x,moving_y,moving_z,fixed_x,fixed_y,fixed_z
0,Pt-0,false,771.75,506.3863636363636,370.3863636363636,Infinity,Infinity,Infinity
1,Pt-1,false,997.8,429.1,300.75,Infinity,Infinity,Infinity
2,Pt-2,false,836.2909700598802,1337.7516167664671,459.71382035928144,Infinity,Infinity,Infinity
3,Pt-3,false,894.59375,810.125,250.375,Infinity,Infinity,Infinity
4,Pt-4,false,906.0117785630154,429.8479387514723,463.68786808009423,Infinity,Infinity,Infinity
5,Pt-5,false,688.1746031746031,1262.031746031746,316.85714285714283,Infinity,Infinity,Infinity
6,Pt-6,false,728.831998212747,986.5829636485495,51.090064787923275,Infinity,Infinity,Infinity
7,Pt-7,false,1231.8165374677003,787.8666666666667,382.0320413436693,Infinity,Infinity,Infinity
8,Pt-8,false,247.32258064516128,622.258064516129,347.06451612903226,Infinity,Infinity,Infinity
9,Pt-9,false,644.4129217142113,1312.753071946507,713.7413486170814,Infinity,Infinity,Infinity


In [105]:
# Create output filename with timestamp
now = datetime.now()
formatted_datetime = now.strftime('%Y%m%d_%H%M%S')
output_file = os.path.join(bigwarp_path, f'{formatted_datetime}_fine_landmarks.csv')

save_merged_landmarks(old_landmarks[old_landmarks["Active"]=="true"], new_landmarks, output_file)

In [22]:

    
# Merge landmarks
merged_file = create_fine_landmarks_file(os.path.join(bigwarp_path, 'rough_landmarks_mov_EM_fix_LM_20220511_RM0008_126hpf_fP10_f2.csv'), 
                                     os.path.join(bigwarp_path,'unmatched_landmarks_mov_EM_fix_LM_20220511_RM0008_126hpf_fP10_f2_part_1.csv'), 
                                     bigwarp_path)

print(f"Merged landmarks saved to: {merged_file}")

Merged landmarks saved to: \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-05-11\f2\bigwarp_alignment\20240909_120239_fine_landmarks.csv


In [12]:
# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Get the sample
sample_id = '20220511_RM0008_126hpf_fP10_f2'
exp = sample_db.get_sample(sample_id)

# Update EM paths
em_stack_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15.tif'  # Replace with actual path
em_mask_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_cp_masks_ft04_cp-3.tif'  # Replace with actual path

exp.em_stack_path = em_stack_path
exp.em_mask_path = em_mask_path

# Save the updated database
sample_db.save(db_path)
print(f"Sample database updated and saved to {db_path}")

# Load EM mask and calculate centroids
em_stack = tifffile.imread(em_stack_path)
em_mask = tifffile.imread(em_mask_path)
em_props = measure.regionprops(em_mask)
em_centroids = np.array([prop.centroid for prop in em_props])

# Create LUT
centroid_lut = pd.DataFrame({
    'id': range(1, len(em_centroids) + 1),
    'em_centroids_zyx_px': [tuple(centroid) for centroid in em_centroids]
})


# Create BigWarp CSV files
bigwarp_path = os.path.join(exp.paths.root_path, "bigwarp_alignment")
os.makedirs(bigwarp_path, exist_ok=True)

# Calculate region properties and centroids
em_props = measure.regionprops(em_mask)
em_centroids = np.array([prop.centroid for prop in em_props])

# Visualization with napari 
viewer = napari.Viewer()
viewer.add_image(em_stack, name='em_stack')
viewer.add_labels(em_mask, name='mask')
viewer.add_points(em_centroids, name='centroids')


Sample database updated and saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv


D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Points layer 'centroids' at 0x261564dfc40>

In [13]:

## Optional Cell Filtering

# Do you want to apply filtering? 
apply_filter = False

if apply_filter:
    # Calculate region properties
    props = measure.regionprops_table(em_mask, em_stack, properties=['label', 'area', 'solidity'])
    data = pd.DataFrame(props)
    data["equivalent_diameter"] = (6 * data["area"] / np.pi) ** (1/3)
    
    # Display summary statistics
    print(data.describe())
    
    ## Visualization
    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    plt.hist(data["area"], bins=100)
    plt.title("Area Distribution")
    plt.subplot(132)
    plt.hist(data["equivalent_diameter"], bins=100)
    plt.title("Equivalent Diameter Distribution")
    plt.subplot(133)
    plt.hist(data["solidity"][data["solidity"] != np.inf], bins=100)
    plt.title("Solidity Distribution")
    plt.tight_layout()
    plt.show()
    

    # Filter cells based on morphological criteria
    cell_filter = data["label"][(data["equivalent_diameter"] > 10) & (data["solidity"] > 0.7) & (data["area"] > 1800)].values.tolist()
    morphology_filter = (data["equivalent_diameter"] > 10) & (data["solidity"] > 0.7) & (data["area"] > 1800)

    # Apply filter to mask
    em_mask_labels_flat = em_mask.flatten()
    em_mask_labels_series = pd.Series(em_mask_labels_flat)
    result = em_mask_labels_series.isin(cell_filter)
    filtered_mask = result.values.reshape(em_mask.shape)

    # Filter centroids
    filtered_centroids = em_centroids[morphology_filter]

    print("Filtering applied.")
    print(f"Original number of cells: {len(em_centroids)}")
    print(f"Number of cells after filtering: {len(filtered_centroids)}")
    
    viewer.add_labels(filtered_mask, name='mask')
    viewer.add_points(filtered_centroids, name='centroids')
else:
    filtered_mask = em_mask
    filtered_centroids = em_centroids
    print("No filtering applied.")


No filtering applied.


In [14]:
new_landmarks = filtered_centroids[:, ::-1]  # Flip coordinates to match coordinate sequence of BigWarp
selected_indices = np.random.choice(len(filtered_centroids), len(filtered_centroids), replace=False)
selected_centroids = new_landmarks[selected_indices].astype(str)
new_landmarks_rnd = selected_centroids.tolist()

create_bigwarp_csv(os.path.join(bigwarp_path, f'unmatched_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv'), new_landmarks_rnd, moving="EM", fixed="LM")
create_bigwarp_csv(os.path.join(bigwarp_path, f'unmatched_landmarks_mov_LM_fix_EM_{exp.sample.id}.csv'), new_landmarks_rnd, moving="LM", fixed="EM")

# Create and save centroids lookup table
centroid_list = [tuple(centroid) for centroid in filtered_centroids]
centroids_lut = pd.DataFrame({
    'id': range(1, len(centroid_list) + 1),
    'em_centroids_zyx_px': centroid_list
})
centroids_lut.to_csv(os.path.join(bigwarp_path, 'lut_centroids.csv'), index=False)

print("All files saved successfully.")

All files saved successfully.


In [15]:
# Define file paths
rough_landmarks_file = os.path.join(bigwarp_path, f'rough_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv')

fine_landmarks_file = os.path.join(bigwarp_path, f'fine_landmarks_mov_EM_fix_LM_{exp.sample.id}.csv')


# Check if rough landmarks files exist
if os.path.exists(rough_landmarks_file):
    print("Rough landmarks files found. Creating fine landmarks files...")
    
    # Create fine landmarks for EM
    create_fine_landmarks(rough_landmarks_file, new_landmarks_rnd, fine_landmarks_file)
    
    
    print("Fine landmarks files created successfully.")
else:
    print("Rough landmarks files not found. Please create rough landmarks files first.")



Rough landmarks files found. Creating fine landmarks files...
Fine landmarks saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-05-11\f2\bigwarp_alignment\fine_landmarks_mov_EM_fix_LM_20220511_RM0008_126hpf_fP10_f2.csv
Fine landmarks files created successfully.
